In [ ]:
# default_exp FLIR_server_utils
# default_cls_lvl 3

# Flir Camera Server
> Utilities for running a multi camera server
>

In [ ]:
#hide
from nbdev.showdoc import *
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

# from  boxfish_stereo_cam.multipyspin import *
import FLIR_pubsub.multi_pyspin as multi_pyspin

import time
import zmq
import threading
import cv2
import imutils

try:
    import mraa
    use_mraa = True
except:
    use_mraa = False
    



In [ ]:
# export 

class CameraThread:
    '''
    Each camera is controlled by a separate thread, allowing it to be started stopped.
    When started the update loop will wait for a camera image and send it the  array data through a zmq socket.
    '''
    def __init__(self, socket_pub, i, yaml_dict):
        self.stopped = True
        self.socket_pub = socket_pub
        self.i = i
        self.yaml_dict = yaml_dict
        self.name = yaml_dict['name']
        self.serial = str(yaml_dict['serial'])
        self.encoding = yaml_dict['encoding']
        self.last_access = time.time()

    def send_array(self, A, flags=0, framedata=None, copy=True, track=False):
        """send a numpy array with metadata"""
        md = dict(
            dtype=str(A.dtype),
            shape=A.shape,
            framedata=framedata,
        )
        self.socket_pub.send_string(self.name, zmq.SNDMORE)
        self.socket_pub.send_json(md, flags | zmq.SNDMORE)
        return self.socket_pub.send(A, flags, copy=copy, track=track)

    def start(self):
        """
        Initialise and set camera thread and begin acquisition
        """
        self.thread = threading.Thread(target=self.update, args=(self.socket_pub, self.i, self.yaml_dict, ))
        self.thread.daemon = False
        self.stopped = False
        self.thread.start()
        return self

    def stop(self):
        """indicate that the thread should be stopped"""
        self.stopped = True
        # wait until stream resources are released (producer thread might be still grabbing frame)
        self.thread.join()

    def update(self, socket, cam_num, yaml_dict):
        # Prepare publisher

        multi_pyspin.init(self.serial)
        multi_pyspin.start_acquisition(self.serial)
        print(f'Starting : {self.name}')
        i = 0
        while True:
            i += 1

            # cam = multi_pyspin._get_cam(serial)
            # image = cam.GetNextImage()
            try:
                image, image_dict = multi_pyspin.get_image(self.serial)
                img = image.GetNDArray()
                shape = img.shape
                if self.encoding is not None:
                    img = cv2.imencode(self.encoding, img)[1]

                md = {'frameid': i, 'encoding': self.encoding, 'size': img.size, 'shape': shape}
                self.send_array( img, framedata=md)
            except Exception as e:
                print(str(e))


            if SHOW_CV_WINDOW:
                if self.encoding is not None:
                    _frame = cv2.imdecode(img, cv2.IMREAD_GRAYSCALE)
                else:
                    _frame = img

                _frame = cv2.cvtColor(_frame, cv2.COLOR_BAYER_BG2BGR)
                _frame = imutils.resize(_frame, width=1000, height=750)
                cv2.imshow(self.name, _frame)
                cv2.waitKey(10)

            if time.time() - self.last_access > 10:
                print(f'Stopping {self.name} due to inactivity.')
                self.stopped = True

            if self.stopped:
                break


        multi_pyspin.end_acquisition(self.serial)
        multi_pyspin.deinit(self.serial)
        

In [ ]:
show_doc(CameraThread.start)
show_doc(CameraThread.stop)

<h4 id="CameraThread.start" class="doc_header"><code>CameraThread.start</code><a href="__main__.py#L29" class="source_link" style="float:right">[source]</a></h4>

> <code>CameraThread.start</code>()

Initialise and set camera thread and begin acquisition

<h4 id="CameraThread.stop" class="doc_header"><code>CameraThread.stop</code><a href="__main__.py#L39" class="source_link" style="float:right">[source]</a></h4>

> <code>CameraThread.stop</code>()

indicate that the thread should be stopped

In [ ]:
# export
class GPIOThread:
    '''
    A thread class to control and toggle Upboard GPIO pins, allowing it to be started & stopped.
    Pin number and frequency can be set
    '''
    def __init__(self, pin_no, freq=2.0):
        self.stopped = True
        # Export the GPIO pin for use
        if use_mraa:
            self.pin = mraa.Gpio(pin_no)
            self.pin.dir(mraa.DIR_OUT)
            self.pin.write(0)
        self.period = 1 / (2 * freq)

    def start(self):
        """Start the pin toggle"""
        self.thread = threading.Thread(target=self.update, args=())
        self.thread.daemon = False
        self.stopped = False
        self.thread.start()
        return self

    def stop(self):
        """Stop the pin toggle"""
        self.stopped = True
        # wait until stream resources are released (producer thread might be still grabbing frame)
        self.thread.join()

    def update(self):
        # Loop
        while True:
            if use_mraa:
                self.pin.write(1)
            # else:
            #     print('1', end='')
            time.sleep(self.period)
            if use_mraa:
                self.pin.write(0)
            # else:
            #     print('0')
            time.sleep(self.period)
            if self.stopped:
                break

In [ ]:
show_doc(GPIOThread.start)
show_doc(GPIOThread.stop)

<h4 id="GPIOThread.start" class="doc_header"><code>GPIOThread.start</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

> <code>GPIOThread.start</code>()

Start the pin toggle

<h4 id="GPIOThread.stop" class="doc_header"><code>GPIOThread.stop</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>GPIOThread.stop</code>()

Stop the pin toggle

In [ ]:
# export
PORT = 5555
SHOW_CV_WINDOW = False

def register():
    """Run multi_pyspin constructor and register multi_pyspin destructor. Should be called once when first imported"""
    multi_pyspin.register() 
    
def server(yaml_dir):
    """
    Main loop for the server. Polls and sets up the cameras. Sets up the socket and port numbers and starts threads.
    """



    # # Install cameras
    pub_threads = []
    yaml_dicts = []
    for i, serial in enumerate(list(multi_pyspin.SERIAL_DICT)):
        print(f'{yaml_dir/serial}.yaml')
        yaml_dict = multi_pyspin.setup(f'{yaml_dir/serial}.yaml')
        yaml_dicts.append(yaml_dict)
             
    # yaml_dir=Path('common')
    # # # Install cameras
    # pub_threads = []
    # yaml_dicts = []
    # for i, serial in enumerate(list(multi_pyspin.SERIAL_DICT)):
    #     yaml_dict = multi_pyspin.setup(f'{yaml_dir/serial}.yaml')
    #     yaml_dicts.append(yaml_dict)


    context = zmq.Context()
    socket_pub = context.socket(zmq.PUB)
    socket_pub.setsockopt(zmq.SNDHWM, 20)
    socket_pub.setsockopt(zmq.LINGER, 0)
    # socket_pub.setsockopt(zmq.SO_REUSEADDR, 1)
    socket_pub.bind(f"tcp://*:{PORT}")

    socket_rep = context.socket(zmq.REP)
    socket_rep.RCVTIMEO = 1000
    socket_rep.bind(f"tcp://*:{PORT+1}")

    for i, yaml_dict in enumerate(list(yaml_dicts)):
        ct = CameraThread(socket_pub, i, yaml_dict)
        ct.start()
        pub_threads.append(ct)

    gpio1 = GPIOThread(29, 2.0).start()
    gpio2 = GPIOThread(31, 10.0).start()
    while True:
        try:
            message = socket_rep.recv().decode("utf-8")
            socket_rep.send_string("OK")
            name = message.split()[1]
            pt = [pt for pt in pub_threads if pt.name == name]
            if len(pt) == 1:
                pt[0].last_access = time.time()
                if pt[0].stopped:
                    pt[0].start()

        except zmq.error.Again:
            pass

        except KeyboardInterrupt:
            break

        except Exception as e:
            print(str(e))

    for ct in pub_threads:
        print(f"stopping {ct.name}")
        ct.stop()

    gpio1.stop()
    gpio2.stop()
    cv2.destroyAllWindows()
    socket_pub.close()
    socket_rep.close()
    context.term()
    
  

## Examples

### Example of a Multiple Camera Server

In [ ]:
from  FLIR_pubsub import FLIR_server_utils
FLIR_server_utils.register()  

from pathlib import Path
if __name__== "__main__":

    yaml_dir= Path.cwd()/'common'
    FLIR_server_utils.server(yaml_dir)

19444712 - connected
19444715 - connected
/home/john/github/FLIR_pubsub/nbs/common/19444712.yaml
19444712 - setting up...
19444712 - executing: "UserSetSelector.SetValue(PySpin.UserSetDefault_Default)"
19444712 - executing: "UserSetLoad.Execute()"
19444712 - executing: "LineSelector.SetValue(PySpin.LineSelector_Line2)"
19444712 - executing: "V3_3Enable.SetValue(True)"
19444712 - executing: "AcquisitionFrameRateEnable.SetValue(False)"
19444712 - executing: "ExposureMode.SetValue(PySpin.ExposureMode_Timed)"
19444712 - executing: "ExposureAuto.SetValue(PySpin.ExposureAuto_Off)"
19444712 - executing: "ExposureTime.SetValue(60000)"
19444712 - executing: "GainSelector.SetValue(PySpin.GainSelector_All)"
19444712 - executing: "GainAuto.SetValue(PySpin.GainAuto_Off)"
19444712 - executing: "Gain.SetValue(6)"
19444712 - executing: "BlackLevelSelector.SetValue(PySpin.BlackLevelSelector_All)"
19444712 - executing: "BlackLevel.SetValue(0)"
19444712 - executing: "GammaEnable.SetValue(True)"
/home/joh

### Camera yaml files

The server polls all installed cameras and trys to match them with corresponding yaml files
Each camera should have a yaml file with the serial number as its file name
The first 3 entries determine serial number and camera name identifiers, and the link encoding of jpeg or not.  
The init section contains the FLIR camera initialisation settings and follows pyspin naming conventions.  

__19312753.yaml__  

```yaml
serial: 19312753 
name: 'FrontRight'  
encoding: '.jpg'  # either null or '.jpg'
init:
    - AcquisitionFrameRateEnable:
        value: True
    - AcquisitionFrameRate:
        value: 2
#    - BinningHorizontal:
#        value: 1
#    - BinningVertical:
#        value: 1
#    - ExposureMode:
#        value: PySpin.ExposureMode_Timed
     - ExposureAuto:
         value: PySpin.ExposureAuto_Continuous
#     - ExposureTime:
#         value: 60000
     - GainSelector:
         value: PySpin.GainSelector_All
     - GainAuto:
         value: PySpin.GainAuto_Off
     - Gain:
         value: 6
     - BlackLevelSelector:
         value: PySpin.BlackLevelSelector_All
     - BlackLevel:
         value: 0
     - GammaEnable:
         value: True
 ```

if you wish to trigger the camera with a hardware digital signal then the following should be used  
```yaml
init:
    - TriggerMode:  
        value: PySpin.TriggerMode_On  
    - TriggerSource:
        value: PySpin.TriggerSource_Line3
    - TriggerOverlap:
        value: PySpin.TriggerOverlap_ReadOut
    - TriggerMode:
        value: PySpin.TriggerMode_On
    - AcquisitionFrameRateEnable:
        value: False   
```

## Service Installation
The server python file __FLIR-server.py__  polls all available cameras and configures them against the yaml file.     
```
from FLIR_pubsub import FLIR_server_utils
FLIR_server_utils.register()  

from pathlib import Path
if __name__== "__main__":

    yaml_dir= Path.cwd()/'common'
    FLIR_server_utils.server(yaml_dir)
```

The server `FLIR-server.py`  requires python > 3.6.  
 Install the following prerequisites
``` 
sudo apt install python3-opencv
pip install opencv-contrib-python
pip install imutils
pip install PyYAML
pip install zmq
```
The server __FLIR-server.py__ is typically run as a service __flir-server.service__.  
To install the service  
`sudo cp flir-server.service /etc/systemd/system/flir-server.service`  

The service can be started and stopped with the following bash commands  

`sudo systemctl start flir-server.service  `  
`sudo systemctl stop flir-server.service  `  

To ensure it runs on boot enable the service  

`sudo systemctl enable flir-server.service` 